<a href="https://colab.research.google.com/github/RodGuarneros/BigData_NLP/blob/main/nlp_hashingTF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the version of spark 3.0 form http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
#spark_version 3.0.2
spark_version = "spark-3.1.1"
os.environ["SPARK_VERSION"] = spark_version

# Install Java, Spark, and Findspark
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bi

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Hashing").getOrCreate()

In [3]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [4]:
# Sample DataFrame with repeating words
dataframe = spark.createDataFrame([
    (0, "The cow cow jumped and jumped cow"),
    (1, "then the cow said"),
    (2, "I am a cow that jumped")
],["id", "words"])

dataframe.show()

+---+--------------------+
| id|               words|
+---+--------------------+
|  0|The cow cow jumpe...|
|  1|   then the cow said|
|  2|I am a cow that j...|
+---+--------------------+



In [5]:
# Tokenize the words
tokenizer = Tokenizer(inputCol="words", outputCol="tokens")
wordsData = tokenizer.transform(dataframe)
wordsData.show()

+---+--------------------+--------------------+
| id|               words|              tokens|
+---+--------------------+--------------------+
|  0|The cow cow jumpe...|[the, cow, cow, j...|
|  1|   then the cow said|[then, the, cow, ...|
|  2|I am a cow that j...|[i, am, a, cow, t...|
+---+--------------------+--------------------+



In [6]:
# Run the hashing term frequency convert words in numbers, the frequency of the word of the collection of the tokens numFeatures: the number of posible numbers you can sasign
hashing = HashingTF(inputCol="tokens", outputCol="hashedValues", numFeatures=pow(2,4))

# Transform into a DF
hashed_df = hashing.transform(wordsData)

In [7]:
# Display new DataFrame
hashed_df.show(truncate=False)

+---+---------------------------------+-----------------------------------------+----------------------------------------+
|id |words                            |tokens                                   |hashedValues                            |
+---+---------------------------------+-----------------------------------------+----------------------------------------+
|0  |The cow cow jumped and jumped cow|[the, cow, cow, jumped, and, jumped, cow]|(16,[1,3,6,11],[1.0,3.0,2.0,1.0])       |
|1  |then the cow said                |[then, the, cow, said]                   |(16,[0,1,3,13],[1.0,1.0,1.0,1.0])       |
|2  |I am a cow that jumped           |[i, am, a, cow, that, jumped]            |(16,[0,3,6,12,13],[1.0,2.0,1.0,1.0,1.0])|
+---+---------------------------------+-----------------------------------------+----------------------------------------+



In [8]:
# Fit the IDF on the data set 
idf = IDF(inputCol="hashedValues", outputCol="features")
idfModel = idf.fit(hashed_df)
rescaledData = idfModel.transform(hashed_df)

In [9]:
# Display the DataFrame
rescaledData.select("words", "features").show(truncate=False)

+---------------------------------+-------------------------------------------------------------------------------------------------------+
|words                            |features                                                                                               |
+---------------------------------+-------------------------------------------------------------------------------------------------------+
|The cow cow jumped and jumped cow|(16,[1,3,6,11],[0.28768207245178085,0.0,0.5753641449035617,0.6931471805599453])                        |
|then the cow said                |(16,[0,1,3,13],[0.28768207245178085,0.28768207245178085,0.0,0.28768207245178085])                      |
|I am a cow that jumped           |(16,[0,3,6,12,13],[0.28768207245178085,0.0,0.28768207245178085,0.6931471805599453,0.28768207245178085])|
+---------------------------------+-------------------------------------------------------------------------------------------------------+

